In [ ]:
import pandas as pd

# Load data
df = pd.read_csv("./IID_Performance.csv", index_col=0)

In [ ]:
df

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Seaborn을 사용하여 bar plot을 그립니다.
# sns.barplot(x='index', y='mIOU (Test)', data=df)
sns.barplot(data=df.transpose())
# # 그래프에 제목을 추가합니다.
# plt.title('Bar Plot')

# # 그래프를 표시합니다.
# plt.show()
# # Plot
# sns.set_theme(style="whitegrid")
# sns.set_context("paper", font_scale=1.5)
# fig, ax = plt.subplots(figsize=(10, 5))
# # ax = sns.barplot(x=df.index, y=["mIOU (Test)", "F1 (Test)", "Acc (Test)"], data=df, palette="Blues_d")
# ax = sns.barplot(x=df.index, y=["mIOU (Test)", "F1 (Test)", "Acc (Test)"], data=df, palette="Blues_d")

In [ ]:
# 샘플 데이터
data = {
    'clients': ['client1', 'client2', 'client3', 'client4', 'client5', 'client6', 'client7', 'client8', 'client9', 'client10'],
    'mIOU': [0.1, 0.5, 0.7, 0.3, 0.6, 0.8, 0.5, 0.4, 0.6, 0.7],
    'F1': [0.5, 0.6, 0.7, 0.8, 0.5, 0.4, 0.6, 0.7, 0.5, 0.6],
    'ACC': [0.9, 0.8, 0.7, 0.6, 0.7, 0.8, 0.9, 0.7, 0.8, 0.6]
}

# DataFrame으로 변환
df = pd.DataFrame(data)
df

In [ ]:
def plot_bargraph(df, save_path, title):
    """
    :param df: DataFrame with columns 'clients', 'mIOU', 'F1', 'Acc'
    """
    # 데이터를 긴 형식으로 변경합니다.
    df_long = pd.melt(df, id_vars=['clients'], value_vars=df.columns[1:])
    
    # 바 그래프 그리기
    plt.figure(figsize=(10, 5))
    sns.barplot(x='clients', y='value', hue='variable', data=df_long)
    
    # x축과 y축 라벨 설정
    # plt.xlabel('Clients')
    plt.ylabel('Scores')
    
    # 그래프 제목 설정
    plt.title(title)
    
    # 범례 추가
    plt.legend(title='Metrics (Test)')
    
    # 그래프 표시
    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.show()


In [ ]:
df

In [ ]:
df = pd.read_csv("./NIID_Performance.csv", index_col=0)
df.reset_index(names='clients', inplace=True)
df.columns = df.columns.str.replace(' (Test)','')
df.drop(columns=['mIOU', 'loss'], inplace=True)
print(df.columns)
plot_bargraph(df, './NIID_Performance.png', 'Segmentation Performance on NIID Dataset')
df = pd.read_csv("./IID_Performance.csv", index_col=0)
df.reset_index(names='clients', inplace=True)
df.columns = df.columns.str.replace(' (Test)','')
df.drop(columns=['mIOU', 'loss'], inplace=True)
print(df.columns)
plot_bargraph(df, './IID_Performance.png', 'Segmentation Performance on IID Dataset')

In [ ]:
# 1. 각각의 Client로컬 학습 모델의 Receptive Field를 계산한다.
# 

import os
import sys
import pathlib
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
import torch
import models
import utils

def load_all_models(path):
    folder_path = path
    file_list = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".pth"):
                file_list.append(pathlib.Path(root) / file)
    # sorting file_list
    file_list.sort()
    # dirnames
    model_state_dict = {}
    for file in file_list:
        print("file_name : ", file)
        # get index from file name ex. "client_0_best_models" > "0", "test 1" > "1"
        if 'global' in file.parent.name:
            model_state_dict[-1] = torch.load(file) 
            print("file.parent.name: ", file.parent.name)
        else:
            index = int(file.parent.name.split('_')[1])
            model_state_dict[index] = torch.load(file)
            print("file.parent.name: ", file.parent.name, "index: ", index)
    return model_state_dict

model_state_dict = load_all_models("/home/suncheol/code/FedTest/0_FedMHAD_Seg/checkpoints/_56121")# 55227")
model_state_dict[-1] = torch.load("/home/suncheol/code/FedTest/0_FedMHAD_Seg/checkpoints/38125/global/model_round99_acc82.04_loss0.64.pth")
model_state_dict[-2] = torch.load("/home/suncheol/code/FedTest/0_FedMHAD_Seg/checkpoints/42301/global/model_round99_acc83.74_loss0.58.pth")

In [ ]:
import datasets
import config
from torch.utils.data import DataLoader

args = config.init_args(server=False, ipykernel=True)
args.output_size = 224
datasetpartition = datasets.PascalVocSegmentationPartition(args)
train_dataset, valid_dataset = datasetpartition.load_partition(-1)
train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=0 )
valid_loader = DataLoader(valid_dataset, batch_size=args.batch_size, shuffle=False, num_workers=0 )
test_loader = valid_loader


In [ ]:
args.device = torch.device("cuda:0" if torch.cuda.is_available() and args.use_cuda else "cpu")

In [ ]:
import torchmetrics
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import segmentation_models_pytorch as smp
import numpy as np
from poutyne import Model

def test(net, testloader, steps: int = None, device: str = "cpu", args=None):
    net = net.to(device)
    
    last_layer_name = list(net.named_children())[-1][0]
    parameters = [
        {'params': [p for n, p in net.named_parameters() if last_layer_name not in n], 'lr': args.learning_rate},
        {'params': [p for n, p in net.named_parameters() if last_layer_name in n], 'lr': args.learning_rate*args.multifly_lr_lastlayer},
    ]
    optimizer = optim.Adam(parameters, lr=args.learning_rate)

    if args.loss_fn == "cross_entropy":
        criterion = nn.CrossEntropyLoss()
    elif args.loss_fn == "dice_loss":
        criterion = smp.losses.DiceLoss(mode = 'multiclass')
    elif args.loss_fn == "focal_loss":
        criterion = smp.losses.FocalLoss(mode = 'multiclass')
    elif args.loss_fn == "LovaszLoss":
        criterion = smp.losses.LovaszLoss(mode = 'multiclass')
    
    from torchmetrics import Accuracy

    def calculate_pixel_accuracy(predictions, targets):
        predictions = predictions.cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()
        """
        Calculates the Pixel Accuracy for Segmentation.
        
        Args:
            predictions (tensor): Predicted segmentation masks.
            targets (tensor): Ground truth segmentation masks.
        
        Returns:
            pixel_accuracy (tensor): Pixel Accuracy for Segmentation.
        """
        accuracy = Accuracy(num_classes=21, task="multiclass")
        pixel_accuracy = accuracy(predictions, targets)
        return pixel_accuracy

    model = Model(
        net,
        optimizer,
        criterion,
        batch_metrics=['accuracy'],
        epoch_metrics=['f1', torchmetrics.JaccardIndex(num_classes=args.num_classes, task="multiclass"),
                       'precision', 'recall'],
        device=device,
    )
    
    logs = model.evaluate_generator(testloader, return_dict_format=True) #, steps=steps)
    
    return logs

In [ ]:
total_logs = {}
args.loss_fn = "cross_entropy"
deeplabv3plus = models.get_network("deeplabv3plus", args.num_classes, args.pretrained, args.excluded_heads)
deeplabv3plus.load_state_dict(model_state_dict[-1])
total_logs[-1] = test(deeplabv3plus, test_loader, device=args.device, args=args)

In [ ]:
args.output_size = 56
datasetpartition = datasets.PascalVocSegmentationPartition(args)
train_dataset, valid_dataset = datasetpartition.load_partition(-1)
train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=0 )
valid_loader = DataLoader(valid_dataset, batch_size=args.batch_size, shuffle=False, num_workers=0 )
test_loader = valid_loader

segformer = models.get_network("segformer", args.num_classes, args.pretrained, args.excluded_heads)
from tqdm import tqdm
for index in tqdm([-2, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]):
    segformer.load_state_dict(model_state_dict[index])
    total_logs[index] = test(segformer, test_loader, device=args.device, args=args)

In [ ]:
total_logs_df = pd.DataFrame(total_logs)
total_logs_df = total_logs_df.transpose()
total_logs_df.reset_index(names='clients', inplace=True)
replace_dict={'-1': 'global (segformer)', '-2': 'global (deeplabv3plus)', '0': 'client 0', '1': 'client 1', '2': 'client 2', '3': 'client 3', '4': 'client 4', '5': 'client 5', '6': 'client 6', '7': 'client 7', '8': 'client 8', '9': 'client 9'}
replace_dict = {int(k): v for k, v in replace_dict.items()}
total_logs_df['clients'] = total_logs_df.clients.apply(lambda x: replace_dict[x])
rename_columns = {'test_loss':'loss', 'test_acc':'acc', 'test_fscore_macro':'f1', 'test_multiclass_jaccard_index':'mIOU', 'test_precision_macro':'precision', 'test_recall_macro':'recall'}
total_logs_df.rename(columns=rename_columns, inplace=True)
total_logs_df.to_csv("./total_logs_NIID.csv", index=False)


In [ ]:
import pandas as pd
total_logs_df = pd.DataFrame(total_logs)
# -2 : global (segformerb0)
# -1 : global (resnet34)
# 0~10 : local clients  

total_logs_df.to_csv("./total_logs_NIID.csv")

In [ ]:
model_state_dict = load_all_models("/home/suncheol/code/FedTest/0_FedMHAD_Seg/checkpoints/_57584")# 55227")
model_state_dict[-1] = torch.load("/home/suncheol/code/FedTest/0_FedMHAD_Seg/checkpoints/17751/global/model_round89_acc82.38_loss0.59.pth")
model_state_dict[-2] = torch.load("/home/suncheol/code/FedTest/0_FedMHAD_Seg/checkpoints/52840/global/model_round78_acc83.25_loss0.62.pth")

In [ ]:
args.output_size = 224
datasetpartition = datasets.PascalVocSegmentationPartition(args)
train_dataset, valid_dataset = datasetpartition.load_partition(-1)
train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=0 )
valid_loader = DataLoader(valid_dataset, batch_size=args.batch_size, shuffle=False, num_workers=0 )
test_loader = valid_loader

total_logs = {}
args.loss_fn = "cross_entropy"
deeplabv3plus = models.get_network("deeplabv3plus", args.num_classes, args.pretrained, args.excluded_heads)
deeplabv3plus.load_state_dict(model_state_dict[-1])
total_logs[-1] = test(deeplabv3plus, test_loader, device=args.device, args=args)

args.output_size = 56
datasetpartition = datasets.PascalVocSegmentationPartition(args)
train_dataset, valid_dataset = datasetpartition.load_partition(-1)
train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=0 )
valid_loader = DataLoader(valid_dataset, batch_size=args.batch_size, shuffle=False, num_workers=0 )
test_loader = valid_loader

segformer = models.get_network("segformer", args.num_classes, args.pretrained, args.excluded_heads)
from tqdm import tqdm
for index in tqdm([-2, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]):
    segformer.load_state_dict(model_state_dict[index])
    total_logs[index] = test(segformer, test_loader, device=args.device, args=args)
    
total_logs_df = pd.DataFrame(total_logs)
total_logs_df = total_logs_df.transpose()
total_logs_df.reset_index(names='clients', inplace=True)
replace_dict={'-1': 'global (segformer)', '-2': 'global (deeplabv3plus)', '0': 'client 0', '1': 'client 1', '2': 'client 2', '3': 'client 3', '4': 'client 4', '5': 'client 5', '6': 'client 6', '7': 'client 7', '8': 'client 8', '9': 'client 9'}
replace_dict = {int(k): v for k, v in replace_dict.items()}
total_logs_df['clients'] = total_logs_df.clients.apply(lambda x: replace_dict[x])
rename_columns = {'test_loss':'loss', 'test_acc':'acc', 'test_fscore_macro':'f1', 'test_multiclass_jaccard_index':'mIOU', 'test_precision_macro':'precision', 'test_recall_macro':'recall'}
total_logs_df.rename(columns=rename_columns, inplace=True)
total_logs_df.to_csv("./total_logs_IID.csv", index=False)


In [ ]:
import pandas as pd

def plot_bargraph(df, save_path, title):
    """
    :param df: DataFrame with columns 'clients', 'mIOU', 'F1', 'Acc'
    """
    # 데이터를 긴 형식으로 변경합니다.
    df_long = pd.melt(df, id_vars=['clients'], value_vars=df.columns[1:])
    sns.color_palette("Set2")
    # 바 그래프 그리기
    plt.figure(figsize=(20, 7))
    ax = sns.barplot(x='clients', y='value', hue='variable', data=df_long, palette="deep")
    # with labels 
    for p in ax.patches:
        ax.annotate("%.2f" % p.get_height(), (p.get_x() + p.get_width() / 2., p.get_height()),
            ha='center', va='center', fontsize=11, xytext=(0, 20),
            textcoords='offset points')
    ax.set_ylim(0.2, 0.9)
    
    # draw text percentile of FL-Segformer vs FL-Deeplabv3+(Rn34) performance
    
    plt.grid(axis='y', linestyle='-', alpha=0.7)
    
    # x축과 y축 라벨 설정
    plt.xlabel('')
    
    # x축 font size 
    plt.xticks(fontsize=12)
    
    plt.ylabel('Scores', fontsize=15)
    #plt.xticks(rotation=20, ha='right', fontsize=12)
    
    # draw grid lines
    plt.grid(axis='y', linestyle='-', alpha=0.4)
    # 그래프 제목 설정
    plt.title(title, fontsize=18)
    
    # 범례 추가
    plt.legend(title='Metrics (Test)', loc='lower right', fontsize=15, title_fontsize=15)
    # 그래프 표시
    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.show()


In [ ]:
total_logs_df = pd.read_csv("./total_logs_IID.csv")
total_logs_df.acc = total_logs_df.acc / 100
total_logs_df = total_logs_df[['clients', 'acc', 'f1', 'precision', 'recall']]
plot_bargraph(total_logs_df, './total_logs_IID.png', 'Segmentation Performance on IID Dataset')
 
total_logs_df = pd.read_csv("./total_logs_NIID.csv")
total_logs_df.acc = total_logs_df.acc / 100
total_logs_df = total_logs_df[['clients', 'acc', 'f1', 'precision', 'recall']]
plot_bargraph(total_logs_df, './total_logs_NIID.png', 'Segmentation Performance on NIID Dataset')